In [1]:
import polars as pl

In [13]:
# Parameters

# Columns
col_stop_t = ['trip_id', 'stop_id']
col_trips = ['route_id', 'trip_id']

# RER B specific
line_b = 'C01743' # RER B's ID_Line
prefix = 'IDFM:' # Prefix for stop_id

In [14]:
# load trips data
trips = pl.read_csv('IDFM-gtfs/trips.txt', separator=',', columns=col_trips)

In [15]:
# filter trips for line B
trips_b = trips.filter(pl.col('route_id') == prefix+line_b)

In [26]:
# load stop_times data
stop_times = pl.read_csv('IDFM-gtfs/stop_times.txt', separator=',', columns=col_stop_t)

In [35]:
# filter stop_id served by line B trips
stop_trips_b = stop_times.join(trips_b, on='trip_id', how='inner')

In [36]:
# drop trip_id column
stop_route_b = stop_trips_b.drop('trip_id')
# drop [stop_id, route_id] duplicates, we are not interested in the time data. We also consider that served stops doesn't change with time
stop_route_b = stop_route_b.unique()

In [51]:
# Extract numbers after the last ":" in stop_id
stop_route_b = stop_route_b.with_columns([pl.col('stop_id').str.split(':').apply(lambda s: s[-1]).alias('stop_id')])

/tmp/ipykernel_32072/524523873.py:2: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  stop_route_b = stop_route_b.with_columns([pl.col('stop_id').str.split(':').apply(lambda s: s[-1]).alias('stop_id')])


In [57]:
%%bash
# get referential stops data : arrets.csv
wget --content-disposition https://data.iledefrance-mobilites.fr/explore/dataset/arrets/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B

--2023-12-04 12:33:46--  https://data.iledefrance-mobilites.fr/explore/dataset/arrets/download/?format=csv
Resolving data.iledefrance-mobilites.fr (data.iledefrance-mobilites.fr)... 34.248.20.69, 34.249.199.226
Connecting to data.iledefrance-mobilites.fr (data.iledefrance-mobilites.fr)|34.248.20.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/csv]
Saving to: ‘arrets.csv’

     0K .......... .......... .......... .......... .......... 77.7K
    50K .......... .......... .......... .......... .......... 1.83M
   100K .......... .......... .......... .......... .......... 55.7K
   150K .......... .......... .......... .......... .......... 2.25M
   200K .......... .......... .......... .......... .......... 2.18M
   250K .......... .......... .......... .......... ..........  108K
   300K .......... .......... .......... .......... .......... 1.98M
   350K .......... .......... .......... .......... .......... 82.7K
   400K .......... 